In [1]:
#Mounting google drive to access train,test data
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Import Relevant Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_columns', None)

In [0]:
#Upload train,test,sample submission dataset files
train_path = "/content/drive/My Drive/data/train.csv"
df_train = pd.read_csv(train_path)

test_path = "/content/drive/My Drive/data/test.csv"
df_test = pd.read_csv(test_path)

sample_submission_path = "/content/drive/My Drive/data/sample_submission.csv"
df_sample_submission = pd.read_csv(sample_submission_path)


In [7]:
#EDA
#Data shape
print("Train data has",df_train.shape[0],"rows and",df_train.shape[1],"columns.")
print("Test data has",df_test.shape[0],"rows and",df_test.shape[1],"columns.")
print("Sample submission data has",df_sample_submission.shape[0],"rows and",df_sample_submission.shape[1],"columns.")

df_train.describe()

#Eye roll the columns
df_train.head(5)
df_test.head(5)
df_sample_submission.head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [0]:
#checking column info
df_train.info()
df_test.info()

#check for missing value in Train dataset
train_null_check = df_train['comment_text'].isnull().sum()
print("train set missing values")
print(train_null_check)
#check for missing value in Test dataset
test_null_check = df_test['comment_text'].isnull().sum()
print("test set missing values")
print(test_null_check)

#drop observations if missing value exists
if train_null_check > 0:
  df_train.dropna(inplace=True)

if test_null_check > 0:
  df_test.dropna(inplace=True)

In [24]:
#check & drop for empty/space strings in 'comment_text' column
def empty_text_check(df):
  for index,id,comment in df.itertuples():  # iterate over the DataFrame
    if type(comment)==str:            # avoid NaN values
      if comment.isspace():         # test 'comment_text' for whitespace
        blanks.append(index)     # add matching index numbers to the list

#train
blanks = []
empty_text_check(df_train[['id','comment_text']])
print(len(blanks), 'train blanks: ', blanks)
if len(blanks) > 0:
  df_train.drop(blanks, inplace=True)

#test
blanks = []
empty_text_check(df_test[['id','comment_text']])
print(len(blanks), 'test blanks: ', blanks)
if len(blanks) > 0:
  df_test.drop(blanks, inplace=True)

print(df_test.shape)
print(df_train.shape)

1 test blanks:  [55142]


In [0]:
#checking imbalance (clean vs non-clean)
#checking the number of observations without any tagging for toxicity (clean) 
rowise_sum = df_train.iloc[:,2:].sum(axis=1)
df_train['clean'] = (rowise_sum==0)
print("Total comments in training set:",len(df_train))
print("Clean comments in training set:",df_train['clean'].sum())

In [0]:
#df_train = df_train.drop('clean',axis=1)
#columnwise label's count analysis
df_train = df_train.drop('clean',axis = 1)
train_label_count = df_train.iloc[:,2:].sum()
train_label_count

plt.figure(figsize=(6,3))
train_count_plot = sns.barplot(train_label_count.index, train_label_count.values,palette="deep")
plt.title("Label Counts")
plt.xlabel("Label")
plt.ylabel("Count")
plt.show()

In [0]:
#Initial observation of dataframe showed that a single observation can have multiple labels also.
#i.e we have multi-label classification, hence check correlation
corr = df_train.corr()
plt.figure(figsize=(10,4))
sns.heatmap(corr,cmap="YlGnBu", xticklabels=corr.columns.values, yticklabels=corr.columns.values, annot=True)

#To do: instead of using the 'corr', we can try crosstab, confusion matrix.

In [0]:
#Feature Engineering
#In multi-label classification we can use Problem transformation technique
#(1) Binary Relevance (2)Classifier Chains (3)Label Powerset 

In [0]:
#Note: For reference we pull out accuracy of each label individually
#and use it as base accuracy (i.e if our model predicted correctly all the tagged labels and failed to predict untagged ones)
target_labels = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
train_base_acc_score = {}
print("Target Labelwise Train Base Accuracy : ")
for i in target_labels:
    acc = ((df_train[i] == 0).sum()/len(df_train[i]))*100
    print(i+':' ,format(acc))
    train_base_acc_score[i] = acc

train_base_acc_score

In [0]:
#defining a function for cleaning of text
def clean_text(original_text):
    cleaned_text = original_text.lower()
    cleaned_text = re.sub(r"\'scuse", " excuse ", cleaned_text)
    cleaned_text = re.sub(r"i'm", "i am ", cleaned_text)
    cleaned_text = re.sub(r"\'ll", " will ", cleaned_text)
    cleaned_text = re.sub(r"what's", "what is ", cleaned_text)
    cleaned_text = re.sub(r"\'s", " ", cleaned_text)
    cleaned_text = re.sub(r"\'ve", " have ", cleaned_text)
    cleaned_text = re.sub(r"can't", "cannot ", cleaned_text)
    cleaned_text = re.sub(r"n't", " not ", cleaned_text)
    cleaned_text = re.sub(r"\'re", " are ", cleaned_text)
    cleaned_text = re.sub(r"\'d", " would ", cleaned_text)
    cleaned_text = re.sub('\W', ' ', cleaned_text)
    cleaned_text = re.sub('\s+', ' ', cleaned_text)
    cleaned_text = cleaned_text.strip(' ')
    return cleaned_text

In [0]:
import re
# cleaning the column comment_text in df_train
cleaned_comment_train = []
for i in range(0,len(df_train)):
    cleaned_comment = clean_text(df_train['comment_text'][i])
    cleaned_comment_train.append(cleaned_comment)
df_train['comment_text'] = pd.Series(cleaned_comment_train).astype(str)

# cleaning the column comment_text in df_test
cleaned_comment_test = []
for i in range(0,len(df_test)):
    cleaned_comment = clean_text(df_test['comment_text'][i])
    cleaned_comment_test.append(cleaned_comment)
df_test['comment_text'] = pd.Series(cleaned_comment_test).astype(str)

In [0]:
#Transforming our input data into a TF-IDF matrix
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=20000,stop_words='english')
vect

In [0]:
#Assigning 'x' for train and test to be used for document term matrix(dtm)
x_train = df_train.comment_text
x_test = df_test.comment_text

In [0]:
#create dtm for train
x_train_dtm = vect.fit_transform(x_train)
x_train_dtm

In [0]:
#create dtm for test
x_test_dtm = vect.transform(x_test)
x_test_dtm

In [0]:
#upto here we have done the required data cleaning and transformation
#now we start with training the model using sklearn library

In [0]:
#(1) Binary Relevance using Logistic Regression
#using sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression(C=12.0)
lr

In [0]:
#using For Loop for binary relevance technique
train_pred_acc_score = {}
for label in target_labels:
  y_train = df_train[label]
  #fitting the model
  lr.fit(x_train_dtm, y_train)
  #predicting on train
  y_pred = lr.predict(x_train_dtm)
  #storing accuracy of training
  train_pred_acc_score[label] = accuracy_score(y_train, y_pred)*100

  #predicting on test
  y_test_pred = lr.predict(x_test_dtm)
  #getting probability
  y_test_probability = lr.predict_proba(x_test_dtm)[:,1]
  #storing in submission dataframe
  df_sample_submission[label] = y_test_probability

In [0]:
#we can compare the train accuracy with reference base accuracy
print("Train prediction accuracy:")
train_pred_acc_score
print("Train base accuracy:")
train_base_acc_score

In [0]:
df_sample_submission.head(2)
df_sample_submission.to_csv('/content/drive/My Drive/data/final_submission_binary_relevance.csv',
                            index = False)

In [0]:
##(2) Classifier Chains using Logistic Regression
# creating a function to add labels/features iterationwise
from scipy.sparse import csr_matrix, hstack
def add_label(x, label):
    
    return hstack([x, csr_matrix(label).T], 'csr')

In [0]:

for label in target_labels:

  y_train = df_train[label]
  #fitting the model
  lr.fit(x_train_dtm, y_train)
  #predicting on train
  y_pred = lr.predict(x_train_dtm)
  #storing accuracy of training
  train_pred_acc_score[label] = accuracy_score(y_train, y_pred)*100
  #predicting on test
  y_test_pred = lr.predict(x_test_dtm)
  #getting probability
  y_test_probability = lr.predict_proba(x_test_dtm)[:,1]
  #storing in submission dataframe
  df_sample_submission[label] = y_test_probability

  #chain current label to X_dtm
  x_train_dtm = add_label(x_train_dtm, y_train)
    
  # chain current label predictions to test_X_dtm
  x_test_dtm = add_label(x_test_dtm, y_test_pred)
  

In [0]:
#check if train accuracy has increased as compared to binary relevance
print("Train prediction accuracy:")
train_pred_acc_score

In [0]:
df_sample_submission.head(2)
df_sample_submission.to_csv('/content/drive/My Drive/data/final_submission_classifier_chain.csv',
                            index = False)

In [0]:
#Model evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

#confusion matrix
print("Confusion Matrix : ")
print(confusion_matrix(y_train, y_pred))
print("Classification Report : ")
print(classification_report(y_train, y_pred))

#AUC score
modelauc_score = roc_auc_score(y_train, y_pred)
print("AUC :",modelauc_score)
# calculate roc curves
lr_fpr, lr_tpr, _ = roc_curve(y_train, y_pred)
# plot the roc curve for the model
plt.plot(lr_fpr, lr_tpr,marker='.',label='Classifier Chain Logistic Regression')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()